In [1]:
import os
import sys
from datetime import datetime
from os.path import abspath
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

from zipline.utils.run_algo import load_extensions
from zipline.data import bundles
from zipline.data.data_portal import DataPortal
from zipline.utils.calendar_utils import get_calendar

from zipline.api import set_max_leverage, schedule_function, set_benchmark,set_commission
from zipline.finance.commission import PerContract, PerDollar, PerShare, PerTrade
from zipline.finance.commission import CommissionModel
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage
from zipline.finance.commission import PerShare, PerTrade, PerDollar
from zipline.api import set_slippage, set_commission
from zipline.data.bundles import register, unregister, ingest
from zipline.data.bundles.csvdir import csvdir_equities
from zipline.utils.calendar_utils import register_calendar, get_calendar
from zipline.api import (order, 
                         order_target,
                         order_value,
                         record, 
                         symbol,
                         get_datetime,
                         order_target_percent,
                         order_target_value,
                         set_benchmark,
                         get_open_orders)
from zipline import run_algorithm
from zipline.utils.calendar_utils import get_calendar
from zipline.api import order_target, record, date_rules, time_rules, symbol # type: ignore


import quantstats as qs
import warnings
warnings.filterwarnings('ignore', category=Warning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)
%matplotlib inline
%load_ext autoreload
%autoreload 2

ROOT_DIR = abspath('../')
sys.path.append(ROOT_DIR)

In [19]:
START_DATE = pd.Timestamp('2013-01-01')
END_DATE = pd.Timestamp('2023-01-01')

BASE_CAPITAL = 100_000

def calculate_years() -> int:
    return int((END_DATE - START_DATE).days / 365)

YEARS = calculate_years()

In [20]:
def plots(results):
    start = results.index[0]
    end = results.index[-1]
    benchmark = pdr.get_data_yahoo('^GSPC', start=start, end=end)['Adj Close'].pct_change()
    results.index = pd.to_datetime(results.index).tz_convert(None)
    results.index = benchmark.index  
    qs.reports.html(results['returns'], benchmark = benchmark, match_dates=True, figsize=(8, 4),
                    title = f"Bollinger Bnad Strategy ADF filter {YEARS} years",
                    output= f"{ROOT_DIR}/outputs/BBstrat_ADF_{YEARS}y.html")

In [21]:
parquet_file_path = f"{ROOT_DIR}/data/clusters/adf_cluster_{YEARS}y.parquet"
data = pd.read_parquet(parquet_file_path)
data = data.iloc[:, 0].tolist()

In [22]:
def initialize(context):
    context.i = 0
    context.tickers = data
    context.bollinger_window = 20
    context.bollinger_dev = 2.0
    context.stop_loss, context.take_profit  = 2.0, 2.0
    
    set_commission(PerShare(cost=0.003))
    set_slippage(VolumeShareSlippage(volume_limit=0.10, price_impact=0.15))

def handle_data(context, data):
    context.i += 1
    if context.i < context.bollinger_window:
        return

    for ticker in context.tickers:
        # Get historical price data using data.history
        prices = data.history(symbol(ticker), 'price', context.bollinger_window + 1, '1d')

        # Calculate Bollinger Bands
        sma = prices.mean()
        rolling_std = prices.std()
        upper_band = sma + (context.bollinger_dev * rolling_std)
        lower_band = sma - (context.bollinger_dev * rolling_std)

        # Get the current price
        current_price = data.current(symbol(ticker), 'price')

        # Check for an existing position
        open_position = context.portfolio.positions[symbol(ticker)].amount

        # Calculate stop-loss and take-profit levels
        stop_loss_price = current_price * (1 - context.stop_loss / 100.0)
        take_profit_price = current_price * (1 + context.take_profit / 100.0)

        # Generate signals based on Bollinger Bands, stop-loss, and take-profit
        if current_price > upper_band and open_position <= 0:
            order_target_percent(symbol(ticker), -0.2)  # Short position
        elif current_price < lower_band and open_position >= 0:
            order_target_percent(symbol(ticker), 0.2)  # Long position
        elif lower_band < current_price < upper_band and open_position != 0:
            # Close position if stop-loss or take-profit conditions are met
            if current_price <= stop_loss_price or current_price >= take_profit_price:
                order_target(symbol(ticker), 0)
                
        # Record the values for later analysis
        record(
            price=current_price,
            upper=upper_band,
            lower=lower_band,
            stop_loss=stop_loss_price,
            take_profit=take_profit_price
        )
        
# Run the algorithm
results = run_algorithm(
    start=START_DATE,
    end=END_DATE,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=BASE_CAPITAL,
    benchmark_returns=None,
    data_frequency='daily',
    bundle='sp500bundle',
)
plots(results)

[*********************100%%**********************]  1 of 1 completed
